In [105]:
import os
import pandas as pd
import subprocess
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.preprocessing import StandardScaler

In [86]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)


In [3]:
os.makedirs('./parts', exist_ok=True)

In [4]:
subprocess.run(['plink', '--bfile', './filtered_data', '--write-snplist', '--out', './temp_snplist'], check=True)

PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./temp_snplist.log.
Options in effect:
  --bfile ./filtered_data
  --out ./temp_snplist
  --write-snplist

16384 MB RAM detected; reserving 8192 MB for main workspace.
591998 variants loaded from .bim file.
1401 people (937 males, 464 females) loaded from .fam.
1401 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1401 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.991679.
591998 variants and 1401 people pass filters and QC.
Among remaining phenotypes, 933 are cases and 468 are controls.
List of variant IDs written to 

CompletedProcess(args=['plink', '--bfile', './filtered_data', '--write-snplist', '--out', './temp_snplist'], returncode=0)

In [5]:
from sklearn.preprocessing import OneHotEncoder

def create_genome(x, a1, a2):
    if x == 0.0:
        return f"{a1}{a1}"
    elif x == 1.0:
        return f"{a1}{a2}"
    elif x == 2.0:
        return f"{a2}{a2}"
    else:
        return f"-"


In [6]:
from tqdm import tqdm

def get_raw_data_parts(part_path, input_plink):
    dfs = []
    with open('./temp_snplist.snplist', 'r') as f:
        all_snps = [line.strip() for line in f]
    total_snps = len(all_snps)
    chunk_size = total_snps // 1000
    if total_snps % 1000 > 0:
        chunk_size += 1  # округляем вверх

    for i in tqdm(range(1000)):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, total_snps)
        
        if start_idx >= total_snps:
            break
        
        part_snps = all_snps[start_idx:end_idx]
        part_name = f"{part_path}/part_{i:03d}"
        
        snp_file = f"{part_name}.snplist"
        with open(snp_file, 'w') as f:
            for snp in part_snps:
                f.write(f"{snp}\n")
        
        subprocess.run([
            'plink',
            '--bfile', input_plink,
            '--extract', snp_file,
            '--make-bed',
            '--out', part_name
        ], check=True, 
        stdout=subprocess.DEVNULL, 
        stderr=subprocess.DEVNULL)
        
        subprocess.run([
            'plink',
            '--bfile', part_name,
            '--recodeA',
            '--out', part_name
        ], check=True, 
        stdout=subprocess.DEVNULL, 
        stderr=subprocess.DEVNULL)
        
        raw_file = f"{part_name}.raw"
        df = pd.read_csv(raw_file, delim_whitespace=True)
        dfs.append(df)

    return dfs

In [87]:
raw_train = get_raw_data_parts("./parts", "all_train")
raw_test = get_raw_data_parts("./parts", "validation")

  0%|          | 0/1000 [00:00<?, ?it/s]Python(66754) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(66755) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
  0%|          | 1/1000 [00:00<12:28,  1.33it/s]Python(66756) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(66757) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
  0%|          | 2/1000 [00:01<12:24,  1.34it/s]Python(66758) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(66759) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
  0%|          | 3/1000 [00:02<11:37,  1.43it/s]Python(66760) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(66761) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
  0%|          | 4/1000 [00:02<1

In [7]:
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)


def get_ohe_data(part_path, plink_train, raw_train, raw_test):
    bim_data = pd.read_csv(f'{plink_train}.bim', sep='\t', header=None, 
                       names=['CHR', 'SNP', 'GEN_DIST', 'BP', 'A1', 'A2'])
    bim_data.set_index('SNP', inplace=True)

    train_ohes = []
    test_ohes = []
    
    for train_part, test_part in tqdm(zip(raw_train, raw_test)):
        columns = []
        for column in train_part.drop(columns=["IID", "PAT", "MAT", "SEX"]).columns[2:]:
            a1 = bim_data.loc[column[:-2], 'A1']
            a2 = bim_data.loc[column[:-2], 'A2']
            df_new = pd.DataFrame()
            df_new[column[:-2]] = train_part[column].apply(lambda x: create_genome(x, a1, a2))
            columns.append(df_new)
        train_letter_part = pd.concat(columns, axis=1)

        columns = []
        for column in test_part.drop(columns=["IID", "PAT", "MAT", "SEX"]).columns[2:]:
            a1 = bim_data.loc[column[:-2], 'A1']
            a2 = bim_data.loc[column[:-2], 'A2']
            df_new = pd.DataFrame()
            df_new[column[:-2]] = test_part[column].apply(lambda x: create_genome(x, a1, a2))
            columns.append(df_new)
        test_letter_part = pd.concat(columns, axis=1)

        enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        train_ohe = enc.fit_transform(train_letter_part)
        columns = enc.get_feature_names_out()
        train_ohe = pd.DataFrame(columns=columns, data=train_ohe)
        test_ohe = enc.transform(test_letter_part)
        test_ohe = pd.DataFrame(columns=columns, data=test_ohe)
        train_ohes.append(train_ohe)
        test_ohes.append(test_ohe)
    train = pd.concat(train_ohes, axis=1)
    test = pd.concat(test_ohes, axis=1)
    y_train = raw_train[0]["PHENOTYPE"]
    y_test = raw_test[0]["PHENOTYPE"]
    return train, y_train, test, y_test

In [88]:
X_train, y_train, X_val, y_val = get_ohe_data("./parts", "all_train", raw_train, raw_test)

1000it [56:32,  3.39s/it]


In [89]:
from sklearn.decomposition import PCA

X_train_parts = []
X_val_parts = []
for i in range((X_train.shape[1] // 200000) + 1):
    pca = PCA(n_components=20)
    X_train_part = pca.fit_transform(X_train.iloc[:, i * 200000:(i + 1) * 200000])
    X_train_parts.append(X_train_part)
    X_val_part = pca.transform(X_val.iloc[:, i * 200000:(i + 1) * 200000])
    X_val_parts.append(X_val_part)
X_train_pca = np.concatenate(X_train_parts, axis=1)
X_val_pca = np.concatenate(X_val_parts, axis=1)


In [90]:
np.savetxt('all_train_pca_n.csv', X_train_pca, delimiter=',', fmt='%.8f')
np.savetxt('validation_pca_n.csv', X_val_pca, delimiter=',', fmt='%.8f')

На следующих 3х ячейках код часто падает из-за отказа ядра. Поэтому результирующие csv были получены запуском итераций отдельно с перезапуском ноутбука между ними.

In [ ]:
X_trains = []
y_trains = []
X_vals = []
y_vals = []
for i in range(1, 6):
    print(1)
    raw_train = get_raw_data_parts("./parts", f"train_{i}")
    raw_test = get_raw_data_parts("./parts", f"test_{i}")
    X_train, y_train, X_val, y_val = get_ohe_data("./parts", f"train_{i}", raw_train, raw_test)
    X_trains.append(X_train)
    y_trains.append(y_train)
    X_vals.append(X_val)
    y_vals.append(y_val)

In [ ]:
from sklearn.decomposition import PCA

X_trains_pca = []
X_vals_pca = []
for i in range(0, 5):
    X_train = X_trains[i]
    y_train = y_trains[i]
    X_val = X_vals[i]
    y_val = y_vals[i]
    X_train_parts = []
    X_val_parts = []
    for i in range((X_train.shape[1] // 200000) + 1):
        pca = PCA(n_components=20)
        X_train_part = pca.fit_transform(X_train.iloc[:, i * 200000:(i + 1) * 200000])
        X_train_parts.append(X_train_part)
        X_val_part = pca.transform(X_val.iloc[:, i * 200000:(i + 1) * 200000])
        X_val_parts.append(X_val_part)
    X_train_pca = np.concatenate(X_train_parts, axis=1)
    X_val_pca = np.concatenate(X_val_parts, axis=1)
    X_trains_pca.append(X_train_pca)
    X_vals_pca.append(X_val_pca)

In [ ]:
np.savetxt('train_1_pca_n.csv', X_trains_pca[0], delimiter=',', fmt='%.8f')
np.savetxt('test_1_pca_n.csv', X_vals_pca[0], delimiter=',', fmt='%.8f')
np.savetxt('train_2_pca_n.csv', X_trains_pca[1], delimiter=',', fmt='%.8f')
np.savetxt('test_2_pca_n.csv', X_vals_pca[1], delimiter=',', fmt='%.8f')
np.savetxt('train_3_pca_n.csv', X_trains_pca[2], delimiter=',', fmt='%.8f')
np.savetxt('test_3_pca_n.csv', X_vals_pca[2], delimiter=',', fmt='%.8f')
np.savetxt('train_4_pca_n.csv', X_trains_pca[3], delimiter=',', fmt='%.8f')
np.savetxt('test_4_pca_n.csv', X_vals_pca[3], delimiter=',', fmt='%.8f')
np.savetxt('train_5_pca_n.csv', X_trains_pca[4], delimiter=',', fmt='%.8f')
np.savetxt('test_5_pca_n.csv', X_vals_pca[4], delimiter=',', fmt='%.8f')

In [117]:
X_train = np.loadtxt(f'./csv/all_train_pca_n.csv', delimiter=',')
y_train = np.array(pd.read_csv(f'all_train.fam', delim_whitespace=True, header=None)[5]) - 1
X_val = np.loadtxt(f'./csv/validation_pca_n.csv', delimiter=',')
y_val = np.array(pd.read_csv(f'validation.fam', delim_whitespace=True, header=None)[5]) - 1

In [118]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)

model_gb = LogisticRegression()
model_gb.fit(X_train, y_train)

y_pred_proba = model_gb.predict_proba(X_val)[:, 1]

# Вычисление и вывод ROC-AUC
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC pca only: {roc_auc:.4f}')

ROC-AUC pca only: 0.4924


In [119]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)

selector = SelectKBest(f_classif, k=5)
X_train_s = selector.fit_transform(X_train, y_train)
X_val_s = selector.transform(X_val)

model_gb = LogisticRegression()
model_gb.fit(X_train_s, y_train)

y_pred_proba = model_gb.predict_proba(X_val_s)[:, 1]

# Вычисление и вывод ROC-AUC
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC pca only: {roc_auc:.4f}')

ROC-AUC pca only: 0.5226


In [120]:
X_trains_pca = []
X_vals_pca = []
for i in range(1, 6):
    X_train_pca = np.loadtxt(f'./csv/train_{i}_pca_n.csv', delimiter=',')
    X_val_pca = np.loadtxt(f'./csv/test_{i}_pca_n.csv', delimiter=',')
    X_trains_pca.append(X_train_pca)
    X_vals_pca.append(X_val_pca)
    y_train = np.array(pd.read_csv(f'train_{i}.fam', delim_whitespace=True, header=None)[5])
    y_test = np.array(pd.read_csv(f'test_{i}.fam', delim_whitespace=True, header=None)[5])
    y_trains.append(y_train)
    y_vals.append(y_test)

In [126]:
rocs = []
for i in range(5):
    X_train_pca = X_trains_pca[i]
    X_val_pca = X_vals_pca[i]
    y_train = y_trains[i] - 1
    y_val = y_vals[i] - 1

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_pca)
    X_val = ss.transform(X_val_pca)

    model_gb = LogisticRegression()
    model_gb.fit(X_train, y_train)

    y_pred_proba = model_gb.predict_proba(X_val)[:, 1]

    # Вычисление и вывод ROC-AUC
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    rocs.append(roc_auc)
    print(f'ROC-AUC pca only: {roc_auc:.4f}')
print(np.mean(rocs))

ROC-AUC pca only: 0.5052
ROC-AUC pca only: 0.3859
ROC-AUC pca only: 0.4945
ROC-AUC pca only: 0.4394
ROC-AUC pca only: 0.3952
0.44403316777653556


In [125]:
rocs = []
for i in range(5):
    X_train_pca = X_trains_pca[i]
    X_val_pca = X_vals_pca[i]
    y_train = y_trains[i] - 1
    y_val = y_vals[i] - 1
    
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train_pca)
    X_val = ss.transform(X_val_pca)

    selector = SelectKBest(f_classif, k=5)
    X_train_new = selector.fit_transform(X_train_pca, y_train)
    X_val_new = selector.transform(X_val_pca)

    model_gb = LogisticRegression()
    model_gb.fit(X_train_new, y_train)

    y_pred_proba = model_gb.predict_proba(X_val_new)[:, 1]

    # Вычисление и вывод ROC-AUC
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    rocs.append(roc_auc)
    print(f'ROC-AUC pca only: {roc_auc:.4f}')
print(np.mean(rocs))

ROC-AUC pca only: 0.4538
ROC-AUC pca only: 0.4337
ROC-AUC pca only: 0.4830
ROC-AUC pca only: 0.4865
ROC-AUC pca only: 0.4668
0.46475819370822435
